<a href="https://colab.research.google.com/github/SidR-13/Winter-Internship/blob/main/Week__5_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web Scraping.

Link for the website : http://books.toscrape.com/

In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [2]:
url = 'http://books.toscrape.com/'
response = requests.get(url)
response

<Response [200]>

In [3]:
response.status_code

200

In [5]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [6]:
type(response.text)

str

In [7]:
soup = bs(response.text)
type(soup)

bs4.BeautifulSoup

In [8]:
soup.find('title').text

'\n    All products | Books to Scrape - Sandbox\n'

In [9]:
soup.find('title').text

'\n    All products | Books to Scrape - Sandbox\n'

In [10]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

Inspecting the HTML tags to identify the book tags

In [11]:
books_tag = soup.find_all('article', class_='product_pod')

In [13]:
len(books_tag)

20

Trying to select a single book and extract all the information regarding that specific book

In [15]:
book_tag = books_tag[0]
book_tag

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

Book Title


The title is represtend in the 'a' tag. We only want the tags with the title attributes.

In [16]:
title_tag = book_tag.find('a', title = True)
title_tag

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [17]:
title_tag.text

'A Light in the ...'

To get the complete title of the book, we use the title attribute.

In [18]:
title_tag['title']

'A Light in the Attic'

In [19]:
#Title
title = book_tag.find('a', title = True)['title']
title

'A Light in the Attic'

In [20]:
#Rating
rating = book_tag.find('p')['class'][1]
rating


'Three'

In [21]:
#Price
price = book_tag.find('p', class_='price_color').text[1:]
price

'£51.77'

In [22]:
#Book link
link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
link

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

Putting the above code into functions so that it can be called.

In [23]:
def get_details(book_tag):
  title = book_tag.find('a', title=True)['title']
  rating = book_tag.find('p')['class'][1]
  price = book_tag.find('p', class_='price_color' ).text[1:]
  link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
  return title, rating, price, link

get_detailes function takes a 'book_tag', extracts all the details abnd then returns it.

In [30]:
def get_soup(url):
  """"Takes the URL and then returns a soup object"""
  resp = requests.get(url)
  if resp.status_code ==200:
    return bs(resp.text)
  else: return None

def get_books(url):
  """"Extract detauils from all of the book tags"""
  soup = get_soup(url)
  book_tags = soup.find_all('article', class_='product_pod')

  books = []
  for books_tag in book_tags:
    books.append(get_details(book_tag))

  return books

In [33]:
url = 'http://books.toscrape.com/'
books = get_books(url)
len(books)

20

In [34]:
books[:3]

[('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html')]

In [42]:
import pandas as pd

def get_all_books(page = 3):
  books = []
  for i in range(1, page+1):
    #This will help when the URL changes
    url = f'http://books.toscrape.com/catalogue/page-{i}.html'
    soup = get_soup(url)
    if soup:
      book_tags = soup.find_all('article', class_='product_pod')

      for book_tag in book_tags:
        books.append(get_details(book_tag))
  books = pd.DataFrame(books, columns = ['title', 'rating', 'price', 'link'])
  return books


Scraping only the first 3 pages

In [43]:
df = get_all_books(3)
df.head()

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...


In [44]:
df.shape

(60, 4)